In [1]:
#Import needed packages
import pandas as pd

In [2]:
#Read in each csv into dataframes
bird_df = pd.read_csv('C:\\Users\\Invario\\Documents\\Wagner\\Data_Analysis-Fall_2019\\Final_Assignment\\Data\\Consolidated_CVS\\bird_consolidated.csv')
bolt_df = pd.read_csv('C:\\Users\\Invario\\Documents\\Wagner\\Data_Analysis-Fall_2019\\Final_Assignment\\Data\\Consolidated_CVS\\bolt_consolidated.csv')
jump_df = pd.read_csv('C:\\Users\\Invario\\Documents\\Wagner\\Data_Analysis-Fall_2019\\Final_Assignment\\Data\\Consolidated_CVS\\jump_consolidated.csv')
lime_df = pd.read_csv('C:\\Users\\Invario\\Documents\\Wagner\\Data_Analysis-Fall_2019\\Final_Assignment\\Data\\Consolidated_CVS\\lime_consolidated.csv')
lyft_df = pd.read_csv('C:\\Users\\Invario\\Documents\\Wagner\\Data_Analysis-Fall_2019\\Final_Assignment\\Data\\Consolidated_CVS\\lyft_consolidated.csv')
skip_df = pd.read_csv('C:\\Users\\Invario\\Documents\\Wagner\\Data_Analysis-Fall_2019\\Final_Assignment\\Data\\Consolidated_CVS\\skip_consolidated.csv')
spin_df = pd.read_csv('C:\\Users\\Invario\\Documents\\Wagner\\Data_Analysis-Fall_2019\\Final_Assignment\\Data\\Consolidated_CVS\\spin_consolidated.csv')

In [3]:
#Select the necessary columns from each dataframe
#Note - Lime includes all of its bikes and scooters and indicated whether it is resrved or not. Will need to clean that data frame further
bird_df_slim = bird_df.loc[:,['bike_id', 'lat', 'lon', 'Month', 'Day', 'Time', 'Company']]
bolt_df_slim = bolt_df.loc[:,['bike_id', 'lat', 'lon', 'Month', 'Day', 'Time', 'Company']]
jump_df_slim = jump_df.loc[:,['bike_id', 'lat', 'lon', 'Month', 'Day', 'Time', 'Company']]
lime_df_slim = lime_df.loc[:,['bike_id', 'lat', 'lon', 'Month', 'Day', 'Time', 'Company','vehicle_type','is_reserved']]
lyft_df_slim = lyft_df.loc[:,['bike_id', 'lat', 'lon', 'Month', 'Day', 'Time', 'Company']]
skip_df_slim = skip_df.loc[:,['bike_id', 'lat', 'lon', 'Month', 'Day', 'Time', 'Company']]
spin_df_slim = spin_df.loc[:,['bike_id', 'lat', 'lon', 'Month', 'Day', 'Time', 'Company']]


In [4]:
#Lime includes both bikes and scooters so I am going to remove the bikes for this analysis
lime_df_slim.head()
lime_df_slim_scoot = lime_df_slim.query("vehicle_type == 'scooter'")
lime_df_slim_scoot_avail = lime_df_slim_scoot.query("is_reserved == '0'")

#Now I can drop the vehicle_type and is_reserved column
lime_df_slim = lime_df_slim_scoot_avail.drop(['vehicle_type','is_reserved'], axis=1)
lime_df_slim.head()

,bike_id,lat,lon,Month,Day,Time,Company
4,NaN,38.9024,-77.0210,11,14,1215PM,Lime.json
5,NaN,38.8974,-76.9469,11,14,1215PM,Lime.json
6,NaN,38.8945,-77.0263,11,14,1215PM,Lime.json
7,NaN,38.8591,-76.9905,11,14,1215PM,Lime.json
8,NaN,38.8862,-77.0165,11,14,1215PM,Lime.json


In [5]:
#Append all the dataframes together to create one large dataframe of data
dc_scooter_data = pd.concat([bird_df_slim, bolt_df_slim, jump_df_slim, lime_df_slim, lyft_df_slim, skip_df_slim, spin_df_slim])


In [6]:
#Now to clean up the date. I want to put it into ISO 8601 format. 
#First I am going to create a dictionary of the potential times and the converted 24 hour format.

time_converted = {
'1200PM': '12:00:00', '1215PM': '12:15:00', '330PM': '15:30:00', '345PM': '15:45:00', 
'630PM': '18:30:00', '830AM': '08:30:00', '930PM': '21:30:00'}

#Next I'll map this dictionary onto the dataframe
#Create a new column, time_converted, that returns the time from the dictionary based on the time string
dc_scooter_data['time_converted'] = dc_scooter_data['Time'].map(time_converted).fillna(dc_scooter_data['Time'])



In [7]:
#Consolidate the month, day and time into ISO 8601 format in a new column
dc_scooter_data = dc_scooter_data.assign(
    date_time = '2019-' + dc_scooter_data.Month.apply(str) + '-' + dc_scooter_data.Day.apply(str) + 'T' + dc_scooter_data.time_converted + '+00:00'
)


#Delete the columns Month, Day, Time
dc_scooter_data = dc_scooter_data.drop(['Month','Day','Time','time_converted'], axis=1)


In [8]:
#Finally I want to remove the ".json" from the company name
#Solution found here: https://stackoverflow.com/questions/13682044/remove-unwanted-parts-from-strings-in-a-column
dc_scooter_data['Company'] = dc_scooter_data['Company'].str.replace(r'.json', '')

dc_scooter_data

,bike_id,lat,lon,Company,date_time
0,ff3b4815-5460-47f1-bd02-d54839f73c3e,38.936881,-77.024106,Bird,2019-11-14T12:15:00+00:00
1,3b4becd9-f2ea-4f82-b7f9-2b996291769c,38.899695,-77.048702,Bird,2019-11-14T12:15:00+00:00
2,76216487-2126-4ad2-8d97-352702739fc3,38.901755,-77.038939,Bird,2019-11-14T12:15:00+00:00
3,9624407c-3982-4139-9d06-2cc51ed68f77,38.899422,-77.049827,Bird,2019-11-14T12:15:00+00:00
4,f5f63231-b63f-4e69-9dbe-e0ee59bf5e3d,38.908696,-77.076372,Bird,2019-11-14T12:15:00+00:00
...,...,...,...,...,...
23020,5011065,38.901080,-77.034760,Spin,2019-11-22T21:30:00+00:00
23021,4405937,38.923260,-77.019600,Spin,2019-11-22T21:30:00+00:00
23022,2761218,38.874560,-77.003920,Spin,2019-11-22T21:30:00+00:00
23023,2630745,38.880730,-77.013250,Spin,2019-11-22T21:30:00+00:00


In [9]:
#Finally same this dataframe as a csv to move to cloud-based environments for the assignment
dc_scooter_data.to_csv('dc_scooter_data.csv')

In [ ]:
#Note - this data includes November 14th and 15th were I did not collect full data and was still deciding on the timing of download.
#I decided not to remove it at this step in case anyone else would like data from those days.